In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Thiết lập style cho biểu đồ
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("✅ Thư viện đã được import thành công!")

## 1. Đọc và Làm Sạch Dữ Liệu

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('/home/jovyan/data/online_retail.csv', encoding='ISO-8859-1')
print(f"📊 Tổng số bản ghi ban đầu: {len(df):,}")
print(f"📝 Các cột trong dữ liệu: {list(df.columns)}")
df.head()

In [ ]:
# Làm sạch dữ liệu
print("🧹 Đang làm sạch dữ liệu...")

# Thông tin ban đầu
print(f"   - Số bản ghi thiếu CustomerID: {df['CustomerID'].isna().sum():,}")
print(f"   - Số bản ghi Quantity <= 0: {(df['Quantity'] <= 0).sum():,}")
print(f"   - Số bản ghi UnitPrice <= 0: {(df['UnitPrice'] <= 0).sum():,}")

# Lọc dữ liệu hợp lệ
df_clean = df[
    (df['CustomerID'].notna()) & 
    (df['Quantity'] > 0) & 
    (df['UnitPrice'] > 0)
].copy()

# Chuyển đổi kiểu dữ liệu
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])
df_clean['CustomerID'] = df_clean['CustomerID'].astype(int)

# Tính tổng tiền
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['UnitPrice']

# Trích xuất thông tin thời gian
df_clean['Year'] = df_clean['InvoiceDate'].dt.year
df_clean['Month'] = df_clean['InvoiceDate'].dt.month
df_clean['Day'] = df_clean['InvoiceDate'].dt.day
df_clean['DayOfWeek'] = df_clean['InvoiceDate'].dt.dayofweek
df_clean['Hour'] = df_clean['InvoiceDate'].dt.hour
df_clean['YearMonth'] = df_clean['InvoiceDate'].dt.to_period('M')

print(f"\n✅ Số bản ghi sau khi làm sạch: {len(df_clean):,}")
print(f"📅 Phạm vi thời gian: {df_clean['InvoiceDate'].min()} đến {df_clean['InvoiceDate'].max()}")
df_clean.head()

In [ ]:
# Thống kê tổng quan
print("="*60)
print("📈 THỐNG KÊ TỔNG QUAN")
print("="*60)
print(f"💰 Tổng doanh thu: £{df_clean['TotalAmount'].sum():,.2f}")
print(f"🧾 Tổng số hóa đơn: {df_clean['InvoiceNo'].nunique():,}")
print(f"👥 Tổng số khách hàng: {df_clean['CustomerID'].nunique():,}")
print(f"📦 Tổng số sản phẩm: {df_clean['StockCode'].nunique():,}")
print(f"🌍 Số quốc gia: {df_clean['Country'].nunique()}")
print(f"💵 Giá trị đơn hàng trung bình: £{df_clean.groupby('InvoiceNo')['TotalAmount'].sum().mean():,.2f}")

## 2. Phân Tích Doanh Thu Theo Thời Gian

In [ ]:
# Doanh thu theo tháng
revenue_monthly = df_clean.groupby('YearMonth')['TotalAmount'].sum().reset_index()
revenue_monthly['YearMonth'] = revenue_monthly['YearMonth'].astype(str)

fig, ax = plt.subplots(figsize=(14, 6))
bars = ax.bar(revenue_monthly['YearMonth'], revenue_monthly['TotalAmount'], color='steelblue', edgecolor='navy')
ax.set_xlabel('Tháng', fontsize=12)
ax.set_ylabel('Doanh thu (£)', fontsize=12)
ax.set_title('📊 DOANH THU THEO THÁNG', fontsize=16, fontweight='bold')
plt.xticks(rotation=45, ha='right')

# Thêm giá trị trên cột
for bar, val in zip(bars, revenue_monthly['TotalAmount']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10000, 
            f'£{val/1000:.0f}K', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/revenue_monthly.png', dpi=150)
plt.show()

print("\n📅 TOP 5 THÁNG DOANH THU CAO NHẤT:")
print(revenue_monthly.nlargest(5, 'TotalAmount').to_string(index=False))

In [ ]:
# Doanh thu theo ngày trong tuần
days = ['Thứ 2', 'Thứ 3', 'Thứ 4', 'Thứ 5', 'Thứ 6', 'Thứ 7', 'CN']
revenue_weekday = df_clean.groupby('DayOfWeek')['TotalAmount'].sum().reset_index()
revenue_weekday['DayName'] = revenue_weekday['DayOfWeek'].map(lambda x: days[x])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Biểu đồ cột
colors = plt.cm.Blues(np.linspace(0.4, 0.9, 7))
axes[0].bar(revenue_weekday['DayName'], revenue_weekday['TotalAmount'], color=colors)
axes[0].set_title('Doanh Thu Theo Ngày Trong Tuần', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Ngày')
axes[0].set_ylabel('Doanh thu (£)')
axes[0].tick_params(axis='x', rotation=45)

# Doanh thu theo giờ
revenue_hour = df_clean.groupby('Hour')['TotalAmount'].sum().reset_index()
axes[1].plot(revenue_hour['Hour'], revenue_hour['TotalAmount'], marker='o', 
             linewidth=2, markersize=8, color='coral')
axes[1].fill_between(revenue_hour['Hour'], revenue_hour['TotalAmount'], alpha=0.3, color='coral')
axes[1].set_title('Doanh Thu Theo Giờ Trong Ngày', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Giờ')
axes[1].set_ylabel('Doanh thu (£)')
axes[1].set_xticks(range(7, 21))

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/revenue_time_patterns.png', dpi=150)
plt.show()

print(f"\n⏰ Giờ cao điểm: {revenue_hour.loc[revenue_hour['TotalAmount'].idxmax(), 'Hour']}h")

## 3. Phân Tích Top Sản Phẩm Bán Chạy

In [ ]:
# Top 15 sản phẩm theo doanh thu
product_revenue = df_clean.groupby(['StockCode', 'Description']).agg({
    'TotalAmount': 'sum',
    'Quantity': 'sum',
    'InvoiceNo': 'nunique'
}).reset_index()
product_revenue.columns = ['StockCode', 'Description', 'Revenue', 'Quantity', 'Orders']
top_products = product_revenue.nlargest(15, 'Revenue')

fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 15))[::-1]
bars = ax.barh(range(len(top_products)), top_products['Revenue'], color=colors)
ax.set_yticks(range(len(top_products)))
ax.set_yticklabels([desc[:40] + '...' if len(desc) > 40 else desc 
                    for desc in top_products['Description']])
ax.invert_yaxis()
ax.set_xlabel('Doanh thu (£)', fontsize=12)
ax.set_title('🏆 TOP 15 SẢN PHẨM CÓ DOANH THU CAO NHẤT', fontsize=16, fontweight='bold')

# Thêm giá trị
for i, (bar, val) in enumerate(zip(bars, top_products['Revenue'])):
    ax.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2, 
            f'£{val:,.0f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/top_products.png', dpi=150)
plt.show()

print("\n📦 TOP 15 SẢN PHẨM BÁN CHẠY NHẤT:")
display(top_products[['Description', 'Revenue', 'Quantity', 'Orders']].head(10))

In [ ]:
# Top sản phẩm theo số lượng bán
top_quantity = product_revenue.nlargest(10, 'Quantity')

fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(range(len(top_quantity)), top_quantity['Quantity'], color='mediumseagreen')
ax.set_yticks(range(len(top_quantity)))
ax.set_yticklabels([desc[:35] + '...' if len(desc) > 35 else desc 
                    for desc in top_quantity['Description']])
ax.invert_yaxis()
ax.set_xlabel('Số lượng bán', fontsize=12)
ax.set_title('📈 TOP 10 SẢN PHẨM THEO SỐ LƯỢNG BÁN', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/top_quantity.png', dpi=150)
plt.show()

## 4. Phân Tích Khách Hàng

In [ ]:
# Phân tích RFM (Recency, Frequency, Monetary)
print("📊 Đang tính toán RFM...")

# Ngày tham chiếu (ngày cuối cùng + 1)
reference_date = df_clean['InvoiceDate'].max() + pd.Timedelta(days=1)

# Tính RFM
rfm = df_clean.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency
    'TotalAmount': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print(f"\n👥 Số khách hàng phân tích: {len(rfm):,}")
print("\n📈 Thống kê RFM:")
display(rfm.describe())

In [ ]:
# Phân loại RFM
def rfm_score(x, column):
    if column == 'Recency':
        if x <= 30: return 5
        elif x <= 90: return 4
        elif x <= 180: return 3
        elif x <= 365: return 2
        else: return 1
    else:  # Frequency, Monetary
        if x <= rfm[column].quantile(0.2): return 1
        elif x <= rfm[column].quantile(0.4): return 2
        elif x <= rfm[column].quantile(0.6): return 3
        elif x <= rfm[column].quantile(0.8): return 4
        else: return 5

rfm['R_Score'] = rfm['Recency'].apply(lambda x: rfm_score(x, 'Recency'))
rfm['F_Score'] = rfm['Frequency'].apply(lambda x: rfm_score(x, 'Frequency'))
rfm['M_Score'] = rfm['Monetary'].apply(lambda x: rfm_score(x, 'Monetary'))
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Phân loại khách hàng
def segment_customer(row):
    if row['R_Score'] >= 4 and row['F_Score'] >= 4 and row['M_Score'] >= 4:
        return 'Champions'
    elif row['R_Score'] >= 4 and row['F_Score'] >= 3:
        return 'Loyal Customers'
    elif row['R_Score'] >= 4:
        return 'Recent Customers'
    elif row['F_Score'] >= 4 or row['M_Score'] >= 4:
        return 'Potential Loyalist'
    elif row['R_Score'] <= 2 and row['F_Score'] <= 2:
        return 'Lost'
    elif row['R_Score'] <= 2:
        return 'At Risk'
    else:
        return 'Need Attention'

rfm['Segment'] = rfm.apply(segment_customer, axis=1)

# Thống kê phân khúc
segment_stats = rfm.groupby('Segment').agg({
    'CustomerID': 'count',
    'Monetary': ['mean', 'sum']
}).round(2)
segment_stats.columns = ['Count', 'Avg_Revenue', 'Total_Revenue']
segment_stats = segment_stats.sort_values('Total_Revenue', ascending=False)

print("\n🎯 PHÂN KHÚC KHÁCH HÀNG:")
display(segment_stats)

In [ ]:
# Biểu đồ phân khúc khách hàng
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart - Số lượng khách hàng theo phân khúc
segment_counts = rfm['Segment'].value_counts()
colors = plt.cm.Set3(np.linspace(0, 1, len(segment_counts)))
axes[0].pie(segment_counts, labels=segment_counts.index, autopct='%1.1f%%', 
            colors=colors, startangle=90, explode=[0.05]*len(segment_counts))
axes[0].set_title('Phân Bố Khách Hàng Theo Phân Khúc', fontsize=14, fontweight='bold')

# Bar chart - Doanh thu theo phân khúc
segment_revenue = rfm.groupby('Segment')['Monetary'].sum().sort_values(ascending=True)
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(segment_revenue)))
axes[1].barh(segment_revenue.index, segment_revenue.values, color=colors)
axes[1].set_xlabel('Tổng Doanh thu (£)')
axes[1].set_title('Doanh Thu Theo Phân Khúc Khách Hàng', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/customer_segments.png', dpi=150)
plt.show()

## 5. Phân Cụm Khách Hàng (K-Means Clustering)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Chuẩn bị dữ liệu cho clustering
rfm_features = rfm[['Recency', 'Frequency', 'Monetary']].copy()

# Xử lý outliers bằng log transform cho Monetary và Frequency
rfm_features['Monetary'] = np.log1p(rfm_features['Monetary'])
rfm_features['Frequency'] = np.log1p(rfm_features['Frequency'])

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

print("✅ Dữ liệu đã được chuẩn hóa cho clustering")

In [ ]:
# Tìm số cluster tối ưu bằng Elbow Method
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    inertias.append(kmeans.inertia_)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
ax.set_xlabel('Số Cluster (K)', fontsize=12)
ax.set_ylabel('Inertia', fontsize=12)
ax.set_title('Elbow Method - Tìm Số Cluster Tối Ưu', fontsize=14, fontweight='bold')
ax.axvline(x=4, color='red', linestyle='--', label='K=4 (đề xuất)')
ax.legend()
plt.tight_layout()
plt.savefig('/home/jovyan/data/output/elbow_method.png', dpi=150)
plt.show()

print("📍 Dựa trên Elbow Method, K=4 được đề xuất")

In [ ]:
# Áp dụng K-Means với K=4
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Thống kê theo cluster
cluster_stats = rfm.groupby('Cluster').agg({
    'CustomerID': 'count',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).round(2)
cluster_stats.columns = ['Count', 'Avg_Recency', 'Avg_Frequency', 'Avg_Monetary']

# Đặt tên cho từng cluster
cluster_names = {
    cluster_stats['Avg_Monetary'].idxmax(): '💎 VIP Customers',
    cluster_stats['Avg_Recency'].idxmax(): '😴 Inactive Customers',
    cluster_stats['Avg_Frequency'].idxmax(): '🔄 Frequent Buyers'
}
for i in range(n_clusters):
    if i not in cluster_names:
        cluster_names[i] = '📊 Regular Customers'

rfm['Cluster_Name'] = rfm['Cluster'].map(cluster_names)
cluster_stats['Cluster_Name'] = cluster_stats.index.map(cluster_names)

print("\n🎯 KẾT QUẢ PHÂN CỤM KHÁCH HÀNG:")
display(cluster_stats)

In [ ]:
# Biểu đồ Scatter 3D cho clustering
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(14, 6))

# 3D scatter plot
ax1 = fig.add_subplot(121, projection='3d')
colors = ['red', 'blue', 'green', 'orange']
for cluster in range(n_clusters):
    cluster_data = rfm[rfm['Cluster'] == cluster]
    ax1.scatter(cluster_data['Recency'], 
                np.log1p(cluster_data['Frequency']), 
                np.log1p(cluster_data['Monetary']),
                c=colors[cluster], label=cluster_names[cluster], alpha=0.6, s=30)
ax1.set_xlabel('Recency')
ax1.set_ylabel('Log(Frequency)')
ax1.set_zlabel('Log(Monetary)')
ax1.set_title('Phân Cụm Khách Hàng (3D)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left', fontsize=8)

# 2D scatter plot
ax2 = fig.add_subplot(122)
for cluster in range(n_clusters):
    cluster_data = rfm[rfm['Cluster'] == cluster]
    ax2.scatter(np.log1p(cluster_data['Frequency']), 
                np.log1p(cluster_data['Monetary']),
                c=colors[cluster], label=cluster_names[cluster], alpha=0.6, s=50)
ax2.set_xlabel('Log(Frequency)', fontsize=12)
ax2.set_ylabel('Log(Monetary)', fontsize=12)
ax2.set_title('Phân Cụm: Frequency vs Monetary', fontsize=12, fontweight='bold')
ax2.legend(fontsize=9)

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/customer_clusters.png', dpi=150)
plt.show()

## 6. Phân Tích Theo Quốc Gia

In [ ]:
# Doanh thu theo quốc gia
country_stats = df_clean.groupby('Country').agg({
    'TotalAmount': 'sum',
    'CustomerID': 'nunique',
    'InvoiceNo': 'nunique'
}).reset_index()
country_stats.columns = ['Country', 'Revenue', 'Customers', 'Orders']
country_stats = country_stats.sort_values('Revenue', ascending=False)

# Top 10 quốc gia (không tính UK)
top_countries = country_stats[country_stats['Country'] != 'United Kingdom'].head(10)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Biểu đồ doanh thu
colors = plt.cm.Spectral(np.linspace(0, 1, 10))
axes[0].barh(top_countries['Country'], top_countries['Revenue'], color=colors)
axes[0].invert_yaxis()
axes[0].set_xlabel('Doanh thu (£)')
axes[0].set_title('Top 10 Quốc Gia (ngoài UK) theo Doanh Thu', fontsize=12, fontweight='bold')

# Pie chart tỷ lệ UK vs Others
uk_revenue = country_stats[country_stats['Country'] == 'United Kingdom']['Revenue'].values[0]
other_revenue = country_stats[country_stats['Country'] != 'United Kingdom']['Revenue'].sum()
axes[1].pie([uk_revenue, other_revenue], labels=['UK', 'Other Countries'], 
            autopct='%1.1f%%', colors=['#3498db', '#e74c3c'], startangle=90,
            explode=[0.05, 0])
axes[1].set_title('Tỷ Lệ Doanh Thu: UK vs Các Nước Khác', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('/home/jovyan/data/output/country_analysis.png', dpi=150)
plt.show()

print(f"\n🇬🇧 Doanh thu UK: £{uk_revenue:,.2f} ({uk_revenue/(uk_revenue+other_revenue)*100:.1f}%)")
print(f"🌍 Doanh thu các nước khác: £{other_revenue:,.2f}")

## 7. Lưu Kết Quả

In [ ]:
import os

# Tạo thư mục output nếu chưa có
output_dir = '/home/jovyan/data/output'
os.makedirs(output_dir, exist_ok=True)

# Lưu các file CSV
rfm.to_csv(f'{output_dir}/customer_rfm_analysis.csv', index=False)
top_products.to_csv(f'{output_dir}/top_products.csv', index=False)
revenue_monthly.to_csv(f'{output_dir}/revenue_monthly.csv', index=False)
country_stats.to_csv(f'{output_dir}/country_stats.csv', index=False)

print("="*60)
print("✅ KẾT QUẢ ĐÃ ĐƯỢC LƯU VÀO THƯ MỤC /home/jovyan/data/output/")
print("="*60)
print("📊 Các file CSV:")
print("   - customer_rfm_analysis.csv (Phân tích RFM & Clustering)")
print("   - top_products.csv (Top sản phẩm bán chạy)")
print("   - revenue_monthly.csv (Doanh thu theo tháng)")
print("   - country_stats.csv (Thống kê theo quốc gia)")
print("\n📈 Các file hình ảnh:")
for f in os.listdir(output_dir):
    if f.endswith('.png'):
        print(f"   - {f}")

## 📋 Tóm Tắt Kết Quả

### 💰 Doanh Thu
- Tháng có doanh thu cao nhất thường là cuối năm (tháng 11-12)
- Ngày trong tuần có doanh thu cao: Thứ 3, Thứ 4, Thứ 5
- Giờ cao điểm: 10h - 14h

### 📦 Sản Phẩm
- Top sản phẩm thường là các mặt hàng quà tặng, đồ trang trí
- Sản phẩm giá rẻ nhưng bán số lượng lớn đóng góp doanh thu đáng kể

### 👥 Khách Hàng
- Phân thành 4 nhóm chính: VIP, Frequent Buyers, Regular, Inactive
- Champions và Loyal Customers chiếm phần lớn doanh thu
- Cần chú ý nhóm "At Risk" để tránh mất khách hàng

### 🌍 Thị Trường
- UK chiếm >80% doanh thu
- Các thị trường tiềm năng: Netherlands, EIRE, Germany, France